In [1]:
!pip install langchain langchain-community langchain_openai langchain_nvidia_ai_endpoints

!pip install sentence-transformers #for text-Embedding

!pip install --upgrade transformers

!pip install pypdf

!pip install boto3

  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
Using cached sentence_transformers-3.4.1-py3-none-any.whl (275 kB)
  Using cached pypdf-5.3.0-py3-none-any.whl.metadata (7.2 kB)
Using cached pypdf-5.3.0-py3-none-any.whl (300 kB)


In [2]:
from langchain.chains import ConversationalRetrievalChain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.embeddings import HuggingFaceEmbeddings 
import torch
from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


To run models from Nvidia go to https://build.nvidia.com/explore/discover

Create an account & generate an API key.


In [3]:
# import getpass
# import os
# from langchain_openai import ChatOpenAI
# from langchain_nvidia_ai_endpoints import ChatNVIDIA

# OpenAI
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# model=ChatOpenAI(model_name="gpt-3.5-turbo")



# nvidia

# os.environ['NVIDIA_API_KEY'] = getpass.getpass("Enter NVIDIA API Key")
# model = ChatNVIDIA(
#   model="meta/llama-3.1-405b-instruct",
#   temperature=0.2,
#   top_p=0.7,
#   max_tokens=1024,
# )

## Nvidia and OpenAI models Usage:
# give some prompt and run below code
# prompt='--'
# response=model.invoke(prompt)

In [4]:
## Bedrock

import json
import boto3


# Creating a bedrock runtime client.

client = boto3.client("bedrock-runtime", region_name="us-east-1") 

# each model has a unique Model Identifier

model_id='anthropic.claude-3-sonnet-20240229-v1:0'



Extracting the content from the Pdf File.

First download the PDF of the syllabus page by clicking print-> Destination: Save as PDF.

Then upload the pdf file to the colab runtime or gdrive.

Update the `pdf_path`.


In [5]:
import boto3

bucket_name = "amazon-sagemaker-058264306111-us-east-1-e23504aef6c5"
object_key = "dzd_5l5kah6gnsnq3r/bzbm82rtfbxpgn/dev/Syllabus.pdf"
local_file_path = "./Syllabus.pdf"  # Temporary storage in SageMaker

# Initialize S3 client (uses the IAM role attached to SageMaker)
s3_client = boto3.client("s3")

# Download the PDF from S3
s3_client.download_file(bucket_name, object_key, local_file_path)
print(f"Downloaded {object_key} to {local_file_path}")

Downloaded dzd_5l5kah6gnsnq3r/bzbm82rtfbxpgn/dev/Syllabus.pdf to ./Syllabus.pdf


In [6]:
pdf_path = "./Syllabus.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

In [7]:
print(documents)

[Document(metadata={'source': './Syllabus.pdf', 'page': 0}, page_content='Course Syllabus\nJump to Today \uea69  Edit\nCIS 509: Analytics for Unstructured Data\nSpring 2025\n \nInstructor of Record\nXiao Liu, Ph.D.\nAssistant Professor\nDepartment of Information Systems\nEmail: xiao.liu.10@asu.edu (mailto:xiao.liu.10@asu.edu)\nOffice: BAC 662\n \nCourse Teaching Aide\nSai Teja Alasyam\nMaster in Computer Science\nEmail: salasya2@asu.edu (mailto:salasya2@asu.edu)\n \nVirtual Office Hours\nTeaching aide:\nTime: TBD\nZoom: TBD\nInstructor: Xiao Liu\nTime: 5:00 pm - 6:00 pm on Friday or by appointment \nZoom: https://asu.zoom.us/my/xiaoliu (https://asu.zoom.us/my/xiaoliu)\n (https://asu.zoom.us/my/xiaoliu)  \n1/9/25, 11:09 PM Syllabus for CIS 509: Analytics Unstructured Data (2025 Spring DYN)\nhttps://canvas.asu.edu/courses/211724/assignments/syllabus 1/11'), Document(metadata={'source': './Syllabus.pdf', 'page': 1}, page_content='Course Description\nThe terms “deep learning” and “Gen AI” 

When you the print the loaded document above you find that it a dictionary of Document objects with page_content split across the indices. We need to seggregate the page_content into one so that model will have full page_content as context.

In [8]:
from langchain.schema import Document

all_page_content = "".join([doc.page_content for doc in documents])

# Create a new Document object with the combined content
documents = [Document(page_content=all_page_content)]
documents

[Document(page_content='Course Syllabus\nJump to Today \uea69  Edit\nCIS 509: Analytics for Unstructured Data\nSpring 2025\n \nInstructor of Record\nXiao Liu, Ph.D.\nAssistant Professor\nDepartment of Information Systems\nEmail: xiao.liu.10@asu.edu (mailto:xiao.liu.10@asu.edu)\nOffice: BAC 662\n \nCourse Teaching Aide\nSai Teja Alasyam\nMaster in Computer Science\nEmail: salasya2@asu.edu (mailto:salasya2@asu.edu)\n \nVirtual Office Hours\nTeaching aide:\nTime: TBD\nZoom: TBD\nInstructor: Xiao Liu\nTime: 5:00 pm - 6:00 pm on Friday or by appointment \nZoom: https://asu.zoom.us/my/xiaoliu (https://asu.zoom.us/my/xiaoliu)\n (https://asu.zoom.us/my/xiaoliu)  \n1/9/25, 11:09 PM Syllabus for CIS 509: Analytics Unstructured Data (2025 Spring DYN)\nhttps://canvas.asu.edu/courses/211724/assignments/syllabus 1/11Course Description\nThe terms “deep learning” and “Gen AI” are among the newest managerial buzzwords. Behind the hype\nassociated with these terms, some leading firms believe they are ex

In [9]:
len(documents)


1

In [10]:
# from bs4 import BeautifulSoup
# from langchain.docstore.document import Document
# def extract_text_from_html(file_path):
#     with open(file_path, "r", encoding="utf-8") as file:
#         soup = BeautifulSoup(file, "html.parser")
#         text = soup.get_text(separator=" ")  # Extract text and separate by spaces
#     return text

# # Step 2: Create a Document object for further processing
# html_path = '/content/Syllabus for CIS 509_ Analytics Unstructured Data (2024 Spring DYN).html'  # Update with your HTML file path
# text = extract_text_from_html(html_path)
# # print(text)
# # Print a sample of the extracted text
# # print("Extracted text:", text[:100000])  # Print the first 500 characters

# # Step 3: Create a Document object
# documents = [Document(page_content=text)]
# len(documents[0].page_content)

In [11]:
"""
Splits documents into smaller chunks using a recursive character-based text splitter.

Parameters:
    chunk_size (int): The maximum size of each text chunk (default is 8000 characters).
    chunk_overlap (int): The number of overlapping characters between consecutive chunks (default is 2000 characters).

Returns:
    list[Document]: A list of chunked document objects, preserving context through overlap.

Usage:
    - This helps in processing large documents by breaking them into smaller, overlapping segments.
    - Useful for retrieval-augmented generation (RAG) and other NLP tasks requiring document chunking.
"""

text_splitter = RecursiveCharacterTextSplitter(chunk_size=8000, chunk_overlap=2000)
chunked_docs = text_splitter.split_documents(documents)

To cover all the relevant context we have set the chunk size to be 8000 and the size of overlapping characters to be 2000. If you find the RAG not reporting correct information consider increasing the size of the chunk size.

For this doc, anything above 8000 chunk size is ideal.

In [12]:
for i, doc in enumerate(chunked_docs):
    print(f"Chunked doc {i+1} length: {len(doc.page_content)}")


Chunked doc 1 length: 7923
Chunked doc 2 length: 7838
Chunked doc 3 length: 7108


Importing the embeddings model `'sentence-transformers/all-mpnet-base-v2'` which transforms the content in our document into embeddings and stores it in Faiss DB.



> Faiss is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM.



In [13]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
faiss_db = FAISS.from_documents(chunked_docs, embeddings)

/tmp/ipykernel_3299/863149276.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
2025-02-26 20:55:27.397342: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 20:55:27.411874: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 20:

Retriever brings out the relevant content from the faiss_db. The search_type is set to "similarity" and search_kwargs is set to 4. So the 4 most similar chunks considered in the context.


In [14]:
"""
Creates a retriever from a FAISS database for similarity-based search.

Parameters:
    search_type (str): The type of search to perform. "similarity" indicates that the retriever will use similarity-based search.
    search_kwargs (dict): Additional search parameters.
        - k (int): The number of top relevant documents to retrieve (default is 2).

Returns:
    BaseRetriever: A retriever instance that can be used to fetch the most relevant documents from the FAISS index.
"""

retriever = faiss_db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 2}
)


https://python.langchain.com/docs/how_to/custom_llm/

Creating a Chain/Pipeline consisting of a retriever.

In [15]:
import json
from typing import Any, Dict, List, Optional
from langchain.llms.base import LLM
from pydantic import Field

class BedrockLLM(LLM):
    """
    Custom LangChain LLM class wrapper for Amazon Bedrock models.
    Inherits from LangChain's base LLM class.
    """
    
    # Define class attributes with Pydantic Fields
    client: Any = Field(...)  # Bedrock client instance
    model_id: str = Field(...)  # Bedrock model identifier
    kwargs: Dict = Field(default_factory=dict)  # Additional model parameters
    
    def __init__(self, client, model_id, **kwargs):
        """
        Initialize the BedrockLLM instance.
        
        Args:
            client: Bedrock client instance
            model_id: Model identifier (e.g., "anthropic.claude-3-sonnet-20240229-v1:0")
            kwargs: Additional keyword arguments for model configuration
        """
        super().__init__()
        self.client = client
        self.model_id = model_id
        self.kwargs = kwargs

    @property
    def _llm_type(self) -> str:
        """
        Return identifier for the LLM type.
        Required by LangChain's LLM interface.
        """
        return "bedrock"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """
        Execute the LLM call to Bedrock.
        
        Args:
            prompt: The input text to send to the model
            stop: Optional list of stop sequences (not implemented)
            
        Returns:
            str: The generated text response
            
        Raises:
            ValueError: If there's an error calling Bedrock or unexpected response format
        """
        # Construct the request payload for Claude 3
        native_request = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 512,
            "temperature": 0.5,
            "messages": [
                {"role": "user", "content": prompt}
            ]
        }
        
        try:
            # Convert the request to JSON string and invoke the model
            request = json.dumps(native_request)
            response = self.client.invoke_model(modelId=self.model_id, body=request)
            
            # Parse the response body from bytes to JSON
            response_body = json.loads(response['body'].read())
            
            # Handle Claude 3's response format which returns content as an array
            if 'content' in response_body:
                content = response_body['content']
                if isinstance(content, list):
                    # Extract and combine all text content from the response
                    text_content = ' '.join(
                        item['text'] for item in content 
                        if item['type'] == 'text'
                    )
                    return text_content
                
            raise ValueError(f"Unexpected response structure: {response_body}")
                
        except Exception as e:
            raise ValueError(f"Error calling Bedrock: {str(e)}")

In [16]:
        
model = BedrockLLM(
    client=client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0"
)

In [17]:
model.invoke("Hi")

'Hello!'

In [18]:
"""
Creates a ConversationalRetrievalChain instance for handling conversational question-answering with retrieval.

Parameters:
    model (LLM): The language model used for generating responses.
    retriever (BaseRetriever): The retriever responsible for fetching relevant documents from a knowledge base.
    return_source_documents (bool, optional): If True, the chain returns the source documents along with the response. Default is True.

Returns:
    ConversationalRetrievalChain: A retrieval-augmented conversational QA chain that can process user queries and fetch relevant documents.
"""

qa_chain = ConversationalRetrievalChain.from_llm(model, retriever,return_source_documents=True)


In [19]:
"""
Interactive QA chatbot using a conversational retrieval chain.

Functions:
    get_user_input():
        - Prompts the user for input.
        - Returns the input as a lowercase string.

    main():
        - Initializes an empty chat history.
        - Collects user input and queries the QA chain.
        - Prints the answer and updates chat history.
        - Supports conversational context by storing previous interactions.

Usage:
    - Run the script to start an interactive QA session.
    - The chatbot retrieves relevant documents and generates responses based on user queries.
"""

import sys

def get_user_input():
    return input('Prompt: ').lower()

def main():
    chat_history = []

    query = get_user_input()

    result = qa_chain.invoke({'question': query, 'chat_history': chat_history})
    print(f'Answer: {result["answer"]}\n')

    chat_history.append((query, result['answer']))

if __name__ == "__main__":
    main()

Prompt:  What weightage does each exam hold in the final grade?


Answer: Unfortunately the syllabus does not explicitly state the weightage or points for any exams. The grading breakdown provided in the syllabus only mentions the following components and their weights:

1. Lab Assignments (6 assignments) - 60% of total grade
2. Course Project 
    - Exploratory Data Analysis - 10 points
    - Project Presentation - 30 points
    - Total Course Project weight - 40% 

The syllabus does not mention any exams or provide weightages for exams in the final grade calculation. If exams are a component, their weightages are not specified in the syllabus information provided.



## Parameters

Performance of the RAG will depend on the Chunk size.
Sometimes the performance will also depend on the `search_kwargs={'k': 4}`.

#### Why we need Chunking?

For larger documents we intend to create a large number of chunks so that the document can fit into our memory.

#### K Nearest Neighbors

K value will determine the amount of relevant information to retrieve. if we set `K = 1`, the retriever will only extract 1 chunk that is most relevant to our prompt.